In [11]:
import re
import glob

import tqdm

import numpy as np
import xarray as xr

In [ ]:
dataset = {
    "id": [],
    "name": [],
    "device": [],
    "signal": []
}


for csv_file in glob.glob("../data/20241223/*.csv"):
    # get device number
    hb_num = re.search("HB-(\d{1,2})", csv_file).group(1)

    # open CSV
    with open(csv_file, "r") as f:
        for line in tqdm.tqdm(f.readlines()):
            # split by |
            buf = line.split("|")

            # if the length is less than 10, this is empty recording
            if len(buf) < 10:
                continue

            # extract signal
            signal = np.array([float(x) for x in buf[6:]]) # / 65535.0 # 16 bit ADC

            # save to dataset
            dataset["id"].append(buf[0])
            dataset["name"].append(buf[1].upper().strip())
            dataset["device"].append(hb_num)
            dataset["signal"].append(signal)

  0%|          | 0/31 [00:00<?, ?it/s]

100%|██████████| 31/31 [00:00<00:00, 1393.68it/s]


In [ ]:
ds = xr.Dataset({
        "signal": (["id", "time"], np.array(dataset["signal"]))
}, coords={
    "id": dataset["id"],
    "name": dataset["name"],
    "device": dataset["device"],
})

ds.to_netcdf(f"../data/baru/merged.nc")
ds

<xarray.Dataset> Size: 8MB
Dimensions:  (id: 120, time: 8800, name: 120, device: 120)
Coordinates:
  * id       (id) <U2 960B '1' '2' '4' '8' '9' '11' ... '26' '27' '28' '30' '31'
  * name     (name) <U22 11kB 'HUSIN ALATAS' 'RENAN' ... 'FIRMANSYAH' 'SAMLAWI'
  * device   (device) <U2 960B '1' '2' '4' '8' '9' ... '26' '27' '28' '30' '31'
Dimensions without coordinates: time
Data variables:
    signal   (id, time) float64 8MB 6.859e+03 6.891e+03 ... 3.815e+03 3.881e+03